In [1]:
import os
os.chdir("../")

In [2]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from src.mlReg.constants import *
from src.mlReg.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = Config_File_Path, 
            params_filepath = Params_File_Path,
            schema_filepath = Schema_File_Path):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [5]:
import os
import zipfile
import gdown

from src.mlReg.utils.logger import logger
from src.mlReg.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                dataset_url = self.config.source_URL
                zip_download_dir = self.config.local_data_file
                logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

                file_id = dataset_url.split("/")[-2]
                prefix = "https://drive.google.com/uc?/export=download&id="
                url = prefix+file_id
                gdown.download(url, zip_download_dir)
            else:
                logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        except Exception as e:
            logger.info(e)
            raise e
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[ [2024-03-06 16:37:15,193] : INFO : common : yaml file: config\config.yaml loaded successfully ]
[ [2024-03-06 16:37:15,539] : INFO : common : yaml file: params.yaml loaded successfully ]
[ [2024-03-06 16:37:15,559] : INFO : common : yaml file: schema.yaml loaded successfully ]
[ [2024-03-06 16:37:15,588] : INFO : common : created directory at: artifacts ]
[ [2024-03-06 16:37:15,599] : INFO : common : created directory at: artifacts/data_ingestion ]
[ [2024-03-06 16:37:15,606] : INFO : 3425504676 : Downloading data from https://drive.google.com/file/d/1hnb2yQ8aowdWGOMJyO0vFsag8K0cVqD7/view?usp=sharing into file artifacts/data_ingestion/gemstone.zip ]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1hnb2yQ8aowdWGOMJyO0vFsag8K0cVqD7
To: f:\Git_Projects\Diamon Price Prediction\artifacts\data_ingestion\gemstone.zip
100%|██████████| 3.06M/3.06M [00:03<00:00, 947kB/s] 
